### Load libraries

In [1]:
# Importing auxiliary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from datetime import datetime

# import pytorch
import torch
import torch.nn as nn
from torch.utils.data import DataLoader


# import tensorboard for logging
from torch.utils.tensorboard import SummaryWriter

# importing local code
import sys
sys.path.append('../')
from datasets import SpectraDataset
from models import FlexibleMLP, FlexibleBackwardMLP
from loops import train_inverse,val_inverse

### Prepare datasets

#### Load data into dataframe and clean them

In [2]:
# filename
filename = '../DL-Assisted-NHA-Inverse-Design-/Dataset 6655.csv'

# load in dataframe
df = pd.read_csv(filename)

# clean and rearrange data
df['Spectra'] = df.values[:,5:][:,::-1].tolist()
df['Spectra'] = df['Spectra'].apply(np.array)
df.drop(df.columns[5:-1], axis=1, inplace=True)
df.columns = ['Lattice','Material','Thickness','Radius','Pitch','Spectra']

#### Split dataframe in train/val and features/labels (X/y) 

In [3]:
# select input features (y)
X_df = df['Spectra']

# select output labels (X)
y_df = df[['Lattice','Material','Thickness','Radius','Pitch']]

# split in training and validation set
test_val_split = 0.1  # portion of data assigned to validation set
X_train, X_val, y_train, y_val = train_test_split(X_df, y_df, test_size=test_val_split, random_state=42)

In [4]:
X_train

4851    [0.0177678, 0.0183766, 0.0188949, 0.0193357, 0...
604     [0.0605793, 0.0606152, 0.05966, 0.0578818, 0.0...
3891    [0.808406, 0.809849, 0.809093, 0.813381, 0.814...
4817    [0.0190139, 0.0184199, 0.0178838, 0.0173684, 0...
4047    [0.0225713, 0.0216366, 0.020792, 0.0200721, 0....
                              ...                        
3772    [0.859133, 0.864281, 0.86973, 0.871492, 0.8704...
5191    [0.0287532, 0.0295561, 0.0304197, 0.0313051, 0...
5226    [0.00877991, 0.00827171, 0.00775168, 0.0072616...
5390    [0.0210184, 0.0225432, 0.0239345, 0.0252061, 0...
860     [0.00322914, 0.003215, 0.00319104, 0.00315318,...
Name: Spectra, Length: 5988, dtype: object

In [5]:
X_val

5896    [0.0211472, 0.0229053, 0.0245804, 0.0261867, 0...
217     [0.0477382, 0.0480398, 0.0481729, 0.0483546, 0...
3214    [0.984692, 0.986608, 0.988734, 0.986776, 0.989...
4516    [0.019098, 0.0185224, 0.0178235, 0.0170732, 0....
1544    [0.0949851, 0.0943849, 0.0933566, 0.0924849, 0...
                              ...                        
425     [0.0150305, 0.0148046, 0.014522, 0.0143151, 0....
4382    [0.0157072, 0.016587, 0.0175634, 0.0185691, 0....
1002    [0.00339143, 0.00332585, 0.00328501, 0.0032520...
681     [0.0225218, 0.022542, 0.0223468, 0.021951, 0.0...
5719    [0.0261101, 0.0269595, 0.0279326, 0.0289608, 0...
Name: Spectra, Length: 666, dtype: object

In [6]:
y_train

,Lattice,Material,Thickness,Radius,Pitch
4851,1,0,135,105,480
604,0,0,125,100,475
3891,0,2,150,50,520
4817,1,0,130,145,475
4047,1,0,100,125,475
...,...,...,...,...,...
3772,0,2,145,55,475
5191,1,0,145,150,475
5226,1,0,150,110,485
5390,1,1,100,130,480


In [7]:
y_val

,Lattice,Material,Thickness,Radius,Pitch
5896,1,1,120,140,480
217,0,0,105,85,520
3214,0,2,120,75,490
4516,1,0,120,115,510
1544,0,1,105,85,500
...,...,...,...,...,...
425,0,0,115,70,515
4382,1,0,115,110,500
1002,0,0,140,60,485
681,0,0,125,80,475


#### Create datasets

In [8]:
# instantiate training and validation dataset
training_dataset = SpectraDataset(X_train,y_train,direction='backward')
val_dataset = SpectraDataset(X_val,y_val,direction='backward')

#### Create dataloaders

In [9]:
# batch size
batch_size = 64

# Create data loaders
train_dataloader = DataLoader(training_dataset, batch_size=batch_size, shuffle= True, pin_memory=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, pin_memory=True)

for X, y in val_dataloader:
    print(f"Shape of X [N, C]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C]: torch.Size([64, 200])
Shape of y: torch.Size([64, 5]) torch.float32


### Training

#### Choosing training device: cpu, gpu, etc...

In [10]:
# Get cpu or gpu device for training.
device = (
    "cuda"
    if torch.cuda.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cuda device


#### Define model

In [11]:
# structure of the neural network
hidden_layers = [200, 500, 500, 500, 500, 500, 2, 3, 3]

# instantiate model
model = FlexibleBackwardMLP(hidden_layers=hidden_layers, activation=nn.GELU(), p=0.1).to(device)

#### Defining loss and optimizer

In [12]:
# base learning rate
lr = 1.1e-4

# defining loss and optimizer
loss_reg = nn.MSELoss()
loss_ce = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr=lr)

#### Run optimization loop

In [13]:
# create timestamp
now = datetime.now()  # current date and time
date_time = now.strftime("%d%m%y_%H%M%S")

# create summary writer for tensorboard
writer_path = '../tb_logs/inverse_' + date_time + '/'
writer = SummaryWriter(writer_path)

# loop over epochs
epochs = 5000
epoch_threshold = 500
save_checkpoint = './best_inverse_model_' + date_time + '.ckpt'
best_loss = 1.0
for epoch in range(epochs):

    # log epoch to console
    print(f"Epoch {epoch+1}\n-------------------------------")

    # performe training and validation loops
    train_loss = train_inverse(train_dataloader, model, loss_reg, loss_ce, optimizer, device)
    val_loss = val_inverse(val_dataloader, model, loss_reg, loss_ce, device)

    # log training and validation loss to console
    writer.add_scalar("Loss/train", train_loss, epoch)
    writer.add_scalar("Loss/val", val_loss, epoch)

    # save checkpoint
    if (val_loss < best_loss) and (epoch>epoch_threshold):
        
        # save checkpoint
        model.train()
        torch.save(model.state_dict(), save_checkpoint)
        best_loss = val_loss

# close connection to tensorboard
writer.flush()
writer.close()

# finished
print("Done!")

Epoch 1
-------------------------------
tensor([[ 1.0743e-03, -1.7510e-03],
        [ 5.4436e-04, -4.4834e-04],
        [-4.1899e-07, -1.8757e-06],
        [-1.2891e-05, -5.9752e-05],
        [ 1.6954e-03, -9.3909e-04],
        [-2.0303e-04,  7.7643e-05],
        [-4.3899e-06, -1.0503e-05],
        [-1.0106e-09,  4.0530e-07],
        [ 5.0736e-05, -1.4764e-05],
        [ 5.5014e-05, -2.9367e-05],
        [ 2.6504e-05, -2.6217e-05],
        [ 3.5299e-05, -1.6995e-05],
        [ 3.3096e-06, -3.2115e-06],
        [ 2.5623e-05, -4.0940e-05],
        [-2.3711e-05, -2.6847e-05],
        [ 4.1972e-05, -1.1248e-05],
        [ 2.2882e-05,  1.2255e-06],
        [ 1.4371e-05, -3.6050e-05],
        [-7.0160e-06,  2.8960e-05],
        [ 2.2769e-05,  3.3133e-05],
        [ 3.9527e-05,  2.6858e-05],
        [ 2.3989e-05, -2.5211e-06],
        [ 5.7543e-06, -2.9620e-06],
        [-2.0750e-05, -3.5393e-06],
        [ 1.0889e-03, -8.1875e-04],
        [ 1.6217e-05, -1.7386e-05],
        [ 3.7058e-05, -6

/opt/conda/conda-bld/pytorch_1720538456841/work/aten/src/ATen/native/cuda/Loss.cu:250: nll_loss_forward_reduce_cuda_kernel_2d: block: [0,0,0], thread: [0,0,0] Assertion `t >= 0 && t < n_classes` failed.
/opt/conda/conda-bld/pytorch_1720538456841/work/aten/src/ATen/native/cuda/Loss.cu:250: nll_loss_forward_reduce_cuda_kernel_2d: block: [0,0,0], thread: [1,0,0] Assertion `t >= 0 && t < n_classes` failed.
/opt/conda/conda-bld/pytorch_1720538456841/work/aten/src/ATen/native/cuda/Loss.cu:250: nll_loss_forward_reduce_cuda_kernel_2d: block: [0,0,0], thread: [4,0,0] Assertion `t >= 0 && t < n_classes` failed.
/opt/conda/conda-bld/pytorch_1720538456841/work/aten/src/ATen/native/cuda/Loss.cu:250: nll_loss_forward_reduce_cuda_kernel_2d: block: [0,0,0], thread: [24,0,0] Assertion `t >= 0 && t < n_classes` failed.
/opt/conda/conda-bld/pytorch_1720538456841/work/aten/src/ATen/native/cuda/Loss.cu:250: nll_loss_forward_reduce_cuda_kernel_2d: block: [0,0,0], thread: [27,0,0] Assertion `t >= 0 && t < n_

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


### Model inference

In [ ]:
# Instantiate inference model and set to evaluation mode
model_inference = FlexibleMLP(hidden_layers=hidden_layers, activation=nn.GELU(), p=0.1).to(
    device)
model_inference.load_state_dict(torch.load(save_checkpoint,weights_only=True))
model_inference.eval()

In [ ]:
# compute inference on all validation samples
X_inference = torch.tensor(X_val.to_numpy().astype(np.float32)).to(device)
y_inference = model_inference(X_inference)
y_true = torch.tensor(np.stack(y_val.to_numpy()).astype(np.float32)).to(device)

# compute normalized loss for each sample in the validation dataset
loss_fn_inference = nn.MSELoss(reduction='none')
with torch.no_grad():
    loss_inference = loss_fn_inference(y_inference,y_true).sum(axis=-1)
norm_mse_discrepancy = loss_inference/((y_true**2).sum(axis=-1))
k_best = torch.argsort(norm_mse_discrepancy)

In [ ]:
n_sample = k_best[70]
plt.plot(y_true[n_sample].cpu().detach())
plt.plot(y_inference[n_sample].cpu().detach())